# Phase 1: Hardware Handshake and Connection Verification

**Objective**: Confirm all API connections, authentication, and software environments are functional.

This notebook implements Phase 1 of the QubitCalibrator agent development:
1. Verify IQM_TOKEN is configured
2. Initialize IQM backend manager
3. Connect to IQM Resonance hardware
4. Retrieve hardware topology
5. Execute a simple "Hello World" circuit

**Success Criteria**: Successfully run a Qiskit circuit on IQM hardware (or emulator) and return measurement counts.

---

**References**:
- Scope of Work: newscopeofwork.md (Part 2, Phase 1)
- IQM Documentation: https://iqm-finland.github.io/iqm-pulla/

## 1. Environment Setup and Imports

In [ ]:
import os
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

# Add src to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

# Import hardware modules
from src.hardware.iqm_backend import IQMBackendManager
from src.agent.tools import AgentTools

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✓ Imports successful")
print(f"Project root: {project_root}")

## 2. Verify IQM Token Configuration

**Security Note**: The IQM_TOKEN should be stored in a `.env` file that is **NOT** committed to version control.

In [ ]:
# Load environment variables
env_path = project_root / '.env'
load_dotenv(env_path)

# Check if token exists (DO NOT PRINT THE TOKEN VALUE)
token_exists = os.getenv('IQM_TOKEN') is not None

if token_exists:
    token_length = len(os.getenv('IQM_TOKEN'))
    print(f"✓ IQM_TOKEN found (length: {token_length} characters)")
    print(f"✓ Token loaded from: {env_path}")
else:
    print("✗ IQM_TOKEN not found!")
    print("Please create a .env file with your IQM API token:")
    print("  IQM_TOKEN=your_token_here")
    raise ValueError("IQM_TOKEN not configured")

## 3. Initialize IQM Backend Manager

The `IQMBackendManager` handles authentication and connection to IQM Resonance.

In [ ]:
try:
    # Initialize backend manager
    backend_manager = IQMBackendManager()
    print(f"✓ IQMBackendManager initialized")
    print(f"  {backend_manager}")
    
except Exception as e:
    print(f"✗ Failed to initialize backend manager: {e}")
    raise

## 4. Test Connection with Emulator (No Credits)

First, we test with the emulator to ensure everything works without consuming hardware credits.

In [ ]:
try:
    # Get emulator backend
    emulator_backend = backend_manager.get_backend(use_emulator=True)
    print(f"✓ Emulator backend obtained")
    print(f"  Backend type: {type(emulator_backend).__name__}")
    print(f"  Backend name: {emulator_backend.name if hasattr(emulator_backend, 'name') else 'N/A'}")
    
except Exception as e:
    print(f"✗ Failed to get emulator backend: {e}")
    print("  This is expected if iqm-pulla[qiskit] is not installed")
    print("  Install with: pip install iqm-pulla[qiskit] qiskit-aer")
    raise

## 5. Execute "Hello World" Circuit on Emulator

Create and run a simple Bell state circuit to verify the execution pipeline.

In [ ]:
from qiskit import QuantumCircuit, transpile

# Create a simple Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)  # Hadamard on qubit 0
qc.cx(0, 1)  # CNOT from qubit 0 to qubit 1
qc.measure([0, 1], [0, 1])  # Measure both qubits

print("Bell State Circuit:")
print(qc.draw(output='text'))

In [ ]:
# Execute on emulator
try:
    # Transpile for backend
    transpiled_qc = transpile(qc, emulator_backend)
    
    # Run with 100 shots
    job = emulator_backend.run(transpiled_qc, shots=100)
    result = job.result()
    counts = result.get_counts()
    
    print("✓ Circuit executed successfully on emulator")
    print(f"  Measurement counts: {counts}")
    print(f"  Total shots: {sum(counts.values())}")
    
    # Expected: ~50% |00⟩ and ~50% |11⟩ for Bell state
    if '00' in counts or '11' in counts:
        print("  ✓ Results look correct for Bell state")
    
except Exception as e:
    print(f"✗ Circuit execution failed: {e}")
    raise

## 6. Test Agent Tools - Hardware Topology (Emulator)

Use the agent tools to retrieve hardware topology via the emulator.

In [ ]:
# Initialize agent tools
tools = AgentTools(backend_manager=backend_manager)
print(f"✓ AgentTools initialized")
print(f"  {tools}")

In [ ]:
# Get hardware topology (using emulator)
topology_result = tools.get_hardware_topology()

print("\nHardware Topology Result:")
print(f"  Status: {topology_result['status']}")
print(f"  Backend: {topology_result['backend_name']}")
print(f"  Number of qubits: {topology_result['n_qubits']}")
print(f"  Available qubits: {topology_result['qubits']}")
print(f"  Connectivity: {topology_result['connectivity']}")

if topology_result['t1_map']:
    print(f"  T1 values (default): {topology_result['t1_map']}")

if topology_result['status'] == 'success':
    print("\n✓ Hardware topology retrieved successfully")
else:
    print(f"\n✗ Failed: {topology_result.get('error', 'Unknown error')}")

## 7. (Optional) Connect to Real Hardware

**WARNING**: This section will consume IQM hardware credits.

Uncomment and run only when you're ready to test on real hardware.

In [ ]:
# UNCOMMENT TO TEST REAL HARDWARE (CONSUMES CREDITS!)

# try:
#     # Get real hardware backend
#     hardware_backend = backend_manager.get_backend(use_emulator=False)
#     print(f"✓ Connected to real hardware: {hardware_backend.name}")
#     
#     # Get topology from real hardware
#     real_topology = backend_manager.get_hardware_topology()
#     print(f"\nReal Hardware Topology:")
#     print(f"  Qubits: {real_topology['qubits']}")
#     print(f"  Connectivity: {real_topology['connectivity']}")
#     
# except Exception as e:
#     print(f"✗ Real hardware connection failed: {e}")
#     print("  This is expected if you don't have valid IQM credits")

In [ ]:
# UNCOMMENT TO RUN BELL STATE ON REAL HARDWARE (CONSUMES CREDITS!)

# try:
#     # Transpile for real hardware
#     transpiled_qc = transpile(qc, hardware_backend)
#     
#     # Run with minimal shots to conserve credits
#     job = hardware_backend.run(transpiled_qc, shots=100)
#     result = job.result()
#     counts = result.get_counts()
#     
#     print("✓ Circuit executed on REAL HARDWARE")
#     print(f"  Measurement counts: {counts}")
#     print(f"  Job ID: {job.job_id() if hasattr(job, 'job_id') else 'N/A'}")
#     
# except Exception as e:
#     print(f"✗ Real hardware execution failed: {e}")

## 8. Verify Connection Method

Use the backend manager's built-in verification.

In [ ]:
# Verify connection (uses real hardware - may consume minimal credits)
# Comment out if you want to avoid any hardware access

# is_connected = backend_manager.verify_connection()
# 
# if is_connected:
#     print("✓ Connection to IQM Resonance verified")
# else:
#     print("✗ Connection verification failed")

## 9. Phase 1 Success Summary

If all cells above executed successfully, Phase 1 is complete!

In [ ]:
print("="*60)
print("PHASE 1: HARDWARE HANDSHAKE - STATUS")
print("="*60)
print()
print("✓ IQM_TOKEN configured")
print("✓ IQMBackendManager initialized")
print("✓ Emulator backend obtained")
print("✓ Bell state circuit executed on emulator")
print("✓ AgentTools initialized")
print("✓ Hardware topology retrieved")
print()
print("Phase 1 Success Criteria: MET ✓")
print()
print("Next Steps:")
print("  → Phase 2: Open-loop characterization (T1, T2, Rabi)")
print("  → Notebook: 02_phase2_characterization.ipynb")
print("="*60)

## Notes and Observations

Add your observations here:

- **IQM Backend**: 
- **Emulator Performance**: 
- **Any Issues**: 
- **Credit Usage** (if real hardware tested): 

---

**End of Phase 1 Notebook**